In [ ]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib.gridspec as gridspec
import math
import uproot
from array import array

print("Got imports")

In [ ]:
DIM= 1

DIMX = 6
DIMY = 5


data_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_stopping_data_nocalib_6k.root"
mc_file = "/Users/alexanderantonakis/Software/SBNDWireModAna/Data/wiremod_ndwidths_stopping_mc_nocalib_6k.root"

print("Got MC and Data Files to compare and analyze")

In [ ]:
dim_dict = {0: "x", 1: "y", 2: "z", 3:"txz", 4:"tyz", 5:"dqdx", 6:"rr"}
label_dict = {0: "Reconstructed Hit X [cm]", 
              1: "Reconstructed Hit Y [cm]", 
              2: "Reconstructed Hit Z [cm]", 
              3:"Reconstructed Hit "+r"$\theta_{XZ}$ [degrees]", 
              4:"Reconstructed Hit "+r"$\theta_{YZ}$ [degrees]", 
              5:"Reconstructed Hit dQ/dx [ADC/cm]",
              6:"Reconstructed Residual Range [cm]"}


xlim_dict = {0:[[-200, 0], [0, 200]],
             1: [[-200, 200], [-200, 200]],
             2: [[0, 500], [0, 500]],
             3:[[-70, 70], [-70, 70]],
             4:[[-70, 70], [-70, 70]],
             5:[[0, 5000], [0, 5000]],
             6:[[0, 600], [0, 600]]

}

In [ ]:
rfile_data = ROOT.TFile.Open(data_file)
rfile_mc = ROOT.TFile.Open(mc_file)

DATA_TAG = "v10_06_00_02"

rfile_data.ls()

In [ ]:
# I want to convert a ROOT TH2D to a plt.hist2d for better plotting
def root_th2d_to_plt_hist2d(th2d):
    # Get the bin edges and contents from the ROOT TH2D
    x_edges = [th2d.GetXaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsX() + 2)]
    y_edges = [th2d.GetYaxis().GetBinLowEdge(i) for i in range(1, th2d.GetNbinsY() + 2)]
    z_values = np.zeros((th2d.GetNbinsX(), th2d.GetNbinsY()))

    for i in range(1, th2d.GetNbinsX() + 1):
        for j in range(1, th2d.GetNbinsY() + 1):
            z_values[i-1, j-1] = th2d.GetBinContent(i, j)

    return np.array(x_edges), np.array(y_edges), np.array(z_values)

def get_1d_edges(h):
    x_edges = [h.GetXaxis().GetBinLowEdge(i) for i in range(1, h.GetNbinsX() + 2)] 
    return x_edges

# Example usage:
# th2d = rfile_data.Get("hwidth0")
# x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(th2d)
# plt.hist2d(x_edges[:-1], y_edges[:-1], z_values.T, bins=[x_edges, y_edges], cmap='viridis')
# plt.colorbar(label='Counts')
# plt.xlabel('X-axis label')                
#plt.ylabel('Y-axis label')
# plt.title('2D Histogram from ROOT TH2D')
# plt.show()    

# I want to convert a ROOT TH1D with errors to a plt.errorbar for better plotting
def root_th1d_to_plt_errorbar(th1d):
    # Get the bin centers, contents, and errors from the ROOT TH1D
    x_values = [th1d.GetBinCenter(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_values = [th1d.GetBinContent(i) for i in range(1, th1d.GetNbinsX() + 1)]
    y_errors = [th1d.GetBinError(i) for i in range(1, th1d.GetNbinsX() + 1)]

    return np.array(x_values), np.array(y_values), np.array(y_errors) 

# Example usage:
# th1d = rfile_data.Get("hwidth0").ProjectionX()
# x_values, y_values, y_errors = root_th1d_to_plt_errorbar(th1d)
# plt.errorbar(x_values, y_values, yerr=y_errors, fmt='o', ecolor='red', capsize=5)
# plt.xlabel('X-axis label')                
# plt.ylabel('Y-axis label')
# plt.title('1D Histogram with Error Bars from ROOT TH1D')
# plt.show()


# Function to create a ROOT TH1D from means and errors
def make_mean_hist(h2d, means, errors):
    h_means = h2d.ProjectionX()
    h_means.Reset()
    for i in range(1, h_means.GetNbinsX() + 1):
        h_means.SetBinContent(i, means[i-1])
        h_means.SetBinError(i, errors[i-1])
    
    return h_means

In [ ]:

def plot_2D_projection(DIMX, DIMY, isData=True, xlim0=None, xlim1=None, ylim0=None, ylim1=None, zlim=None, DATA_TAG=None):

    kNtpcs = 2
    kNplanes = 3

    fig, axs = plt.subplots(kNtpcs, kNplanes, figsize=(12, 8), sharey=True)      
    gs = gridspec.GridSpec(1, 2 + 1, width_ratios=[1, 1, 0.05], wspace=0.1)

    largest_max_z = 0
    smallest_z = 100000

    for tpc in range(kNtpcs):
        for plane in range(kNplanes):
            idx = 3*tpc + plane
            #thn = rfile_data.Get("hwidth"+str(idx))

            htemp = 0
            if isData:
                htemp = rfile_data.Get("hwidth"+str(idx))
            else:
                htemp = rfile_mc.Get("hwidth"+str(idx))

           
            h = htemp.Projection(DIMY, DIMX)

            x_edges, y_edges, z_values = root_th2d_to_plt_hist2d(h)
            z_masked = np.ma.masked_where(z_values == 0, z_values)

            cmap = plt.cm.viridis.copy()
            cmap.set_bad(color='white')

            ax = axs[tpc, plane]
            X, Y = np.meshgrid(x_edges, y_edges, indexing='ij')  # Needed for pcolormesh
            #ax.pcolormesh(X, Y, z_values, shading='auto', cmap='viridis')
            MAX_Z = np.max(z_values)
            MIN_Z = np.min(z_values[(z_values > 0)])
            if MIN_Z < smallest_z:
                smallest_z = MIN_Z

            if MAX_Z > largest_max_z:
                largest_max_z = MAX_Z
            if zlim is not None:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', vmin=zlim[0], vmax=zlim[1])
            else:
                pcm = ax.pcolormesh(X, Y, z_masked, cmap=cmap, shading='auto', vmin=MIN_Z, vmax=MAX_Z)

            if tpc == 0:
                if xlim0 is not None:
                    ax.set_xlim(xlim0)
                if ylim0 is not None:
                    ax.set_ylim(ylim0)
            else:
                if xlim1 is not None:
                    ax.set_xlim(xlim1)
                if ylim1 is not None:
                    ax.set_ylim(ylim1)

            fig.colorbar(pcm, ax=ax) 
            #im = ax.hist2d(x_edges, y_edges, z_values.T, cmap='viridis')
            ax.set_title(f'TPC {tpc}, Plane {plane}')
            #fig.colorbar(im[3], ax=ax, label='')
          
            h.Delete()
            

    print("Largest max z value:", largest_max_z)
    print("Smallest z", smallest_z)
    fig.supxlabel(label_dict[DIMX], fontsize=20)
    fig.supylabel(label_dict[DIMY], fontsize=20)
    t = "SBND Off-Beam "
    if isData:
        t += "Data: "
    else:
        t += "MC: "
    if DATA_TAG is not None:
        t += DATA_TAG
    fig.suptitle(t, fontsize=20)
    #cbar_ax = fig.add_subplot(gs[0, 2])
    #fig.colorbar(pcm, cax=cbar_ax)
    #fig.colorbar(pcm, ax=axs, fraction=.05, label='')
    plt.tight_layout()
    plt.show()

print("Defining plot function")

In [ ]:
plot_2D_projection(DIMX, DIMY, isData=True, xlim0=None, xlim1=None, ylim0=None, ylim1=None, zlim=None, DATA_TAG=DATA_TAG)

In [ ]:
plot_2D_projection(DIMX, DIMY, isData=False, xlim0=None, xlim1=None, ylim0=None, ylim1=None, zlim=None, DATA_TAG=DATA_TAG)